In [2]:
# example of training an conditional gan on the fashion mnist dataset
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Concatenate
 
# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1), n_classes=12):
	# label input
	in_label = Input(shape=(1,))
	# embedding for categorical input
	li = Embedding(n_classes, 50)(in_label)
	# scale up to image dimensions with linear activation
	n_nodes = in_shape[0] * in_shape[1]
	li = Dense(n_nodes)(li)
	# reshape to additional channel
	li = Reshape((in_shape[0], in_shape[1], 1))(li)
	# image input
	in_image = Input(shape=in_shape)
	# concat label as a channel
	merge = Concatenate()([in_image, li])
	# downsample
	fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(merge)
	fe = LeakyReLU(alpha=0.2)(fe)
	# downsample
	fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
	fe = LeakyReLU(alpha=0.2)(fe)
	# flatten feature maps
	fe = Flatten()(fe)
	# dropout
	fe = Dropout(0.4)(fe)
	# output
	out_layer = Dense(1, activation='sigmoid')(fe)
	# define model
	model = Model([in_image, in_label], out_layer)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model
 
# define the standalone generator model
def define_generator(latent_dim, n_classes=10):
	# label input
	in_label = Input(shape=(1,))
	# embedding for categorical input
	li = Embedding(n_classes, 50)(in_label)
	# linear multiplication
	n_nodes = 7 * 7
	li = Dense(n_nodes)(li)
	# reshape to additional channel
	li = Reshape((7, 7, 1))(li)
	# image generator input
	in_lat = Input(shape=(latent_dim,))
	# foundation for 7x7 image
	n_nodes = 128 * 7 * 7
	gen = Dense(n_nodes)(in_lat)
	gen = LeakyReLU(alpha=0.2)(gen)
	gen = Reshape((7, 7, 128))(gen)
	# merge image gen and label input
	merge = Concatenate()([gen, li])
	# upsample to 14x14
	gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(merge)
	gen = LeakyReLU(alpha=0.2)(gen)
	# upsample to 28x28
	gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
	gen = LeakyReLU(alpha=0.2)(gen)
	# output
	out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)
	# define model
	model = Model([in_lat, in_label], out_layer)
	return model
 
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# get noise and label inputs from generator model
	gen_noise, gen_label = g_model.input
	# get image output from the generator model
	gen_output = g_model.output
	# connect image output and label input from generator as inputs to discriminator
	gan_output = d_model([gen_output, gen_label])
	# define gan model as taking noise and label and outputting a classification
	model = Model([gen_noise, gen_label], gan_output)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model
 
# load fashion mnist images
def load_real_samples():
	# load dataset
	(trainX, trainy), (_, _) = load_data()
	# expand to 3d, e.g. add channels
	X = expand_dims(trainX, axis=-1)
	# convert from ints to floats
	X = X.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return [X, trainy]
 
# # select real samples
def generate_real_samples(dataset, n_samples):
	# split into images and labels
	images, labels = dataset
	# choose random instances
	ix = randint(0, images.shape[0], n_samples)
	# select images and labels
	X, labels = images[ix], labels[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return [X, labels], y
 
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=10):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = randint(0, n_classes, n_samples)
	return [z_input, labels]
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	z_input, labels_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	images = generator.predict([z_input, labels_input])
	# create class labels
	y = zeros((n_samples, 1))
	return [images, labels_input], y
 
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128):
	bat_per_epo = int(dataset[0].shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			[X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
			# generate 'fake' examples
			[X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
			# prepare points in latent space as input for the generator
			[z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
			# summarize loss on this batch
			print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
	# save the generator model
	g_model.save('cgan_generator.h5')
 
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

C:\Users\Oliver\anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\Oliver\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


>1, 1/468, d1=0.682, d2=0.694 g=0.693
>1, 2/468, d1=0.605, d2=0.699 g=0.688
>1, 3/468, d1=0.543, d2=0.710 g=0.677
>1, 4/468, d1=0.486, d2=0.727 g=0.662
>1, 5/468, d1=0.434, d2=0.754 g=0.641
>1, 6/468, d1=0.390, d2=0.784 g=0.619
>1, 7/468, d1=0.352, d2=0.814 g=0.603
>1, 8/468, d1=0.337, d2=0.838 g=0.601
>1, 9/468, d1=0.322, d2=0.829 g=0.629
>1, 10/468, d1=0.310, d2=0.770 g=0.704
>1, 11/468, d1=0.323, d2=0.687 g=0.797
>1, 12/468, d1=0.307, d2=0.591 g=0.928
>1, 13/468, d1=0.314, d2=0.506 g=1.013
>1, 14/468, d1=0.323, d2=0.458 g=1.078
>1, 15/468, d1=0.297, d2=0.469 g=1.034
>1, 16/468, d1=0.300, d2=0.526 g=0.923
>1, 17/468, d1=0.293, d2=0.598 g=0.813
>1, 18/468, d1=0.292, d2=0.658 g=0.735
>1, 19/468, d1=0.200, d2=0.717 g=0.683
>1, 20/468, d1=0.209, d2=0.808 g=0.618
>1, 21/468, d1=0.180, d2=0.905 g=0.558
>1, 22/468, d1=0.132, d2=0.987 g=0.528
>1, 23/468, d1=0.140, d2=0.994 g=0.546
>1, 24/468, d1=0.104, d2=0.905 g=0.631
>1, 25/468, d1=0.095, d2=0.750 g=0.789
>1, 26/468, d1=0.079, d2=0.561 g=0

>1, 209/468, d1=0.682, d2=0.808 g=0.656
>1, 210/468, d1=0.678, d2=0.789 g=0.710
>1, 211/468, d1=0.665, d2=0.755 g=0.734
>1, 212/468, d1=0.688, d2=0.728 g=0.798
>1, 213/468, d1=0.628, d2=0.640 g=0.846
>1, 214/468, d1=0.635, d2=0.639 g=0.858
>1, 215/468, d1=0.616, d2=0.629 g=0.852
>1, 216/468, d1=0.636, d2=0.646 g=0.863
>1, 217/468, d1=0.628, d2=0.664 g=0.800
>1, 218/468, d1=0.634, d2=0.743 g=0.772
>1, 219/468, d1=0.612, d2=0.782 g=0.707
>1, 220/468, d1=0.644, d2=0.808 g=0.693
>1, 221/468, d1=0.644, d2=0.796 g=0.717
>1, 222/468, d1=0.654, d2=0.784 g=0.737
>1, 223/468, d1=0.697, d2=0.750 g=0.788
>1, 224/468, d1=0.711, d2=0.665 g=0.844
>1, 225/468, d1=0.717, d2=0.628 g=0.886
>1, 226/468, d1=0.755, d2=0.610 g=0.926
>1, 227/468, d1=0.750, d2=0.578 g=0.955
>1, 228/468, d1=0.750, d2=0.552 g=0.970
>1, 229/468, d1=0.730, d2=0.538 g=1.016
>1, 230/468, d1=0.775, d2=0.533 g=0.988
>1, 231/468, d1=0.720, d2=0.555 g=0.984
>1, 232/468, d1=0.664, d2=0.557 g=0.965
>1, 233/468, d1=0.733, d2=0.608 g=0.914


>1, 414/468, d1=0.672, d2=0.651 g=0.779
>1, 415/468, d1=0.704, d2=0.663 g=0.775
>1, 416/468, d1=0.678, d2=0.662 g=0.771
>1, 417/468, d1=0.629, d2=0.646 g=0.778
>1, 418/468, d1=0.741, d2=0.655 g=0.770
>1, 419/468, d1=0.679, d2=0.647 g=0.795
>1, 420/468, d1=0.667, d2=0.665 g=0.787
>1, 421/468, d1=0.650, d2=0.636 g=0.786
>1, 422/468, d1=0.653, d2=0.622 g=0.792
>1, 423/468, d1=0.647, d2=0.640 g=0.798
>1, 424/468, d1=0.692, d2=0.641 g=0.794
>1, 425/468, d1=0.646, d2=0.658 g=0.778
>1, 426/468, d1=0.665, d2=0.652 g=0.799
>1, 427/468, d1=0.687, d2=0.663 g=0.769
>1, 428/468, d1=0.686, d2=0.666 g=0.756
>1, 429/468, d1=0.662, d2=0.652 g=0.751
>1, 430/468, d1=0.664, d2=0.677 g=0.760
>1, 431/468, d1=0.666, d2=0.672 g=0.760
>1, 432/468, d1=0.680, d2=0.650 g=0.748
>1, 433/468, d1=0.664, d2=0.676 g=0.741
>1, 434/468, d1=0.660, d2=0.665 g=0.740
>1, 435/468, d1=0.714, d2=0.673 g=0.764
>1, 436/468, d1=0.656, d2=0.667 g=0.757
>1, 437/468, d1=0.679, d2=0.681 g=0.743
>1, 438/468, d1=0.670, d2=0.668 g=0.749


>2, 154/468, d1=0.606, d2=0.699 g=0.730
>2, 155/468, d1=0.629, d2=0.691 g=0.736
>2, 156/468, d1=0.617, d2=0.691 g=0.740
>2, 157/468, d1=0.612, d2=0.675 g=0.740
>2, 158/468, d1=0.627, d2=0.684 g=0.759
>2, 159/468, d1=0.604, d2=0.659 g=0.764
>2, 160/468, d1=0.591, d2=0.674 g=0.775
>2, 161/468, d1=0.612, d2=0.694 g=0.785
>2, 162/468, d1=0.588, d2=0.666 g=0.778
>2, 163/468, d1=0.590, d2=0.659 g=0.769
>2, 164/468, d1=0.594, d2=0.668 g=0.761
>2, 165/468, d1=0.614, d2=0.671 g=0.798
>2, 166/468, d1=0.588, d2=0.672 g=0.747
>2, 167/468, d1=0.581, d2=0.684 g=0.759
>2, 168/468, d1=0.601, d2=0.703 g=0.721
>2, 169/468, d1=0.607, d2=0.720 g=0.730
>2, 170/468, d1=0.625, d2=0.701 g=0.739
>2, 171/468, d1=0.613, d2=0.693 g=0.732
>2, 172/468, d1=0.593, d2=0.680 g=0.761
>2, 173/468, d1=0.608, d2=0.667 g=0.758
>2, 174/468, d1=0.607, d2=0.654 g=0.777
>2, 175/468, d1=0.639, d2=0.630 g=0.810
>2, 176/468, d1=0.620, d2=0.667 g=0.792
>2, 177/468, d1=0.626, d2=0.647 g=0.815
>2, 178/468, d1=0.619, d2=0.642 g=0.783


KeyboardInterrupt: 